In [1]:
import tensorflow as tf

In [2]:
( x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape( -1, 784  )
x_test = x_test.reshape( -1, 784 )

x_train = tf.cast( x_train, tf.float32 )
x_test = tf.cast( x_test, tf.float32 )

y_train = tf.keras.utils.to_categorical( y_train, 10 )
y_test = tf.keras.utils.to_categorical( y_test, 10 )

print( x_train.shape, y_train.shape )

(60000, 784) (60000, 10)


In [3]:
w1 = tf.Variable( tf.random.truncated_normal( [ 784, 30 ] ) )
b1 = tf.Variable( tf.random.truncated_normal( [ 1, 30 ] ) )

w2 = tf.Variable( tf.random.truncated_normal( [ 30, 10 ] ) )
b2 = tf.Variable( tf.random.truncated_normal( [ 1, 10 ] ) )

In [4]:
def sigma( X ):
    return tf.divide( tf.constant( 1.0 ),
                  tf.add( tf.constant( 1.0 ), tf.exp( -X ) ) )

def sigma_prime( X ):
    return sigma( X ) * ( 1. - sigma( X ) )

In [5]:
l1 = lambda X: tf.add( tf.matmul( X, w1 ), b1 )
a1 = lambda X: sigma( l1( X ) )
l2 = lambda X: tf.add( tf.matmul( a1( X ), w2 ), b2 )
y_pred = lambda X: sigma( l2( X ) )

In [6]:
diff = lambda X, Y: y_pred( X ) - Y

d_l2 = lambda X, Y: diff( X, Y ) * sigma_prime( l2( X ) )
d_b2 = lambda X, Y: d_l2( X, Y )
d_w2 = lambda X, Y: tf.matmul( tf.transpose( a1( X ) ), d_l2( X, Y ) )

d_a1 = lambda X, Y: tf.matmul( d_l2( X, Y ), tf.transpose( w2 ) )
d_l1 = lambda X, Y: d_a1( X, Y ) * sigma_prime( l1( X ) )
d_b1 = lambda X, Y: d_l1( X, Y )
d_w1 = lambda X, Y: tf.matmul( tf.transpose( X ), d_l1( X, Y ) )

In [7]:
learning_rate = 0.5
def Step( X, Y ):
    return ( [
    w1.assign( w1 - learning_rate * d_w1( X, Y ) ),
    b1.assign( b1 - learning_rate *
              tf.reduce_mean( d_b1( X, Y ) ) ),
    w2.assign( w2 - learning_rate * d_w2( X, Y ) ),
    b2.assign( b2 - learning_rate *
              tf.reduce_mean( d_b2( X, Y ) ) ),
] )

def Acct( X, Y ):
    acct_mat = tf.equal( tf.argmax( y_pred( X ), 1 ), tf.argmax( Y, 1 ) )
    
    return ( tf.reduce_sum( tf.cast( acct_mat, tf.float32 ) ) )

In [8]:
for i in range( 10000 ):
    Step( x_train, y_train )
    
    if ( i % 500 == 0 ):
        res = Acct( x_test[ : 1000 ], y_test[ : 1000 ] )
        tf.print( res )

85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
85
